# What about the LLMs?

LLMs are reputed to have revolutionised automatic language processing. Since the introduction of BERT-type models, all language processing applications have been based on LLMs, of varying degrees of sophistication and size. These models are trained on multiple tasks and are therefore capable of performing new tasks without learning, simply from a prompt. This is known as "zero-shot learning" because there is no learning phase as such. We are going to test these models on our classification task.

Huggingface is a Franco-American company that develops tools for building applications based on Deep Learning. In particular, it hosts the huggingface.co portal, which contains numerous Deep Learning models. These models can be used very easily thanks to the [Transformer] library (https://huggingface.co/docs/transformers/quicktour) developed by HuggingFace.

Using a transform model in zero-shot learning with HuggingFace is very simple: [see documentation](https://huggingface.co/tasks/zero-shot-classification)

However, you need to choose a suitable model from the list of models compatible with Zero-Shot classification. HuggingFace offers [numerous models](https://huggingface.co/models?pipeline_tag=zero-shot-classification). 

The classes proposed to the model must also provide sufficient semantic information for the model to understand them.

**Question**:

* Write a code to classify an example of text from an article in Le Monde using a model transformed using zero-shot learning with the HuggingFace library.
* choose a model and explain your choice
* choose a formulation for the classes to be predicted
* show that the model predicts a class for the text of the article (correct or incorrect, analyse the results)
* evaluate the performance of your model on 100 articles (a test set).
* note model sizes, processing times and classification results


Notes :
* make sure that you use the correct Tokenizer when using a model 
* start testing with a small number of articles and the first 100's of characters for faster experiments.

In [1]:
# Choisir modèle multilingue, essayer avec un petit modèle et avec un grand.

In [2]:
import pandas as pd

In [3]:
 articles_df = pd.read_csv('LeMonde2003_9classes.csv.gz')

In [4]:
# Filter out the UNE class
articles_df = articles_df[articles_df['category'] != 'UNE']

In [5]:
sample = articles_df.sample(100)

In [6]:
sample

,text,category
2273,jean-max séry ancien conseiller municipal vert...,SOC
23866,le conseil d'état a donné raison fin décembre ...,SPO
9215,florian chekler un homme apparemment déséquili...,SOC
21307,une odeur âcre nauséabonde signale qu'on appro...,FRA
8158,l'éditeur français de logiciels d'aide à la dé...,ENT
...,...,...
24469,a l'occasion de la cérémonie des voeux des aut...,SOC
1777,le ministre dominicain des relations extérieur...,INT
8712,conjoncture on aura beaucoup de mal à atteindr...,SOC
5668,la précédente tournée de bruce springsteen le ...,ART


In [7]:
texts = sample.text

In [8]:
articles_df.category.unique()

array(['SPO', 'ART', 'FRA', 'SOC', 'INT', 'ENT'], dtype=object)

In [9]:
candidate_labels = ['companies', 'international', 'arts', 'society', 'France', 'sports', 'font page articles']

# Dictionnaire de correspondance
corresp_dict = {
    'companies':'ENT',
    'international':'INT',
    'arts':'ART',
    'society':'SOC',
    'France':'FRA',
    'sports':'SPO',
    'font page articles':'UNE'
}

In [10]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 36.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 71.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 50.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 75.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 78.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 80.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 74.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 72.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [11]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 38.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 85.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 84.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2


In [12]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.0 MB/s eta 0:00:00


In [13]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.2 MB/s eta 0:00:00


In [16]:
import torch
import tensorflow as tf
from transformers import pipeline

AttributeError: module 'numpy' has no attribute '_no_nep50_warning'

In [ ]:
import time

In [ ]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
from tqdm import tqdm

In [ ]:
predictions = []
scores = []

# Mesure du temps de traitement
start_time = time.time()
for text in tqdm(texts.to_list(), desc="Traitement", unit="texte"):
    result = classifier(text[:100], candidate_labels, multi_label=True)
    
    # Trouver l'indice du score le plus élevé
    max_score_index = result['scores'].index(max(result['scores']))

    # Sélectionner le label correspondant et son score
    selected_label = result['labels'][max_score_index]
    selected_score = result['scores'][max_score_index]

    # Ajouter les éléments aux listes
    predictions.append(selected_label)
    scores.append(selected_score)
    
# Temps de traitement
end_time = time.time()
processing_time = end_time - start_time

# Taille du modèle
model_dir = os.path.join(os.path.expanduser('~'), '.cache', 'huggingface', 'transformers', 'facebook', 'bart-large-mnli')
model_size = sum(os.path.getsize(os.path.join(model_dir, f)) for f in os.listdir(model_dir) if os.path.isfile(os.path.join(model_dir, f)))

# Afficher les résultats
print(f"Time processing: {processing_time:.4f} seconds")
print(f"Model size: {model_size / (1024**2):.2f} MB")

sample['prediction'] = predictions
sample['scores'] = scores

In [ ]:
sample

In [76]:
# Remplacer les valeurs dans la colonne en utilisant le dictionnaire
sample['prediction'] = sample['prediction'].replace(corresp_dict)

KeyError: 'prediction'

In [77]:
# Calculer le nombre de classifications correctes
count_equal = (sample['category'] == sample['prediction']).sum()
total_lines = len(sample)
accuracy_percentage = (count_equal / total_lines) * 100

KeyError: 'prediction'

In [78]:
print(f"L'évaluation des performances du modèle montre que 'category' et 'prediction' sont égales dans {count_equal} cas sur {total_lines} lignes, soit un taux de précision de {accuracy_percentage:.2f}%")

NameError: name 'count_equal' is not defined